In [3]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import r2_score, mean_squared_error as MSE
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

Создадим новый pipeline для модели, которую будем экспортировать в Fast API сервис. В этот pipeline включим:
- приведение признаков `mileage`, `engine`, `max_power` и `torque` к числовому типу
- заполнение пропусков в значениях признаков медианой
- удаление дубликатов
- удаление признака `selling_price` (поскольку изначально он есть в тестовой выборке)
- преобразование признака `name` к более простому виду
- кодирование категориальных признаков при помоще one-hot-encoding
- стандартизация числовых признаков

In [25]:
class CustomTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.medians = {}
    
    def fit(self, df, y=None):
       return self
        

    def transform(self, df):
        """
        Преобразует данные тестового набора.
        """
        df = df.copy()        
        self.delete_duplicates(df)
        self.name_processing(df)
        self.name_to_categorical(df)
        return df
    
    @staticmethod
    def delete_duplicates(df):
        df.drop_duplicates(inplace=True)
    
    @staticmethod
    def name_processing(df):
        df['name'] = df['name'].apply(lambda x: ' '.join(x.split()[:2]))

    @staticmethod
    def name_to_categorical(df):
        name_counts = df['name'].value_counts()
        rare_names = name_counts[name_counts < 100].index
        df['name'] = df['name'].replace(rare_names, 'rare')


In [6]:
df_train = pd.read_csv('https://raw.githubusercontent.com/Murcha1990/MLDS_ML_2022/main/Hometasks/HT1/cars_train.csv')
df_test = pd.read_csv('https://raw.githubusercontent.com/Murcha1990/MLDS_ML_2022/main/Hometasks/HT1/cars_test.csv')

In [19]:
# Для того, чтобы найти медианное значение для столбцов, содержащих пропуски, нам нужно сначала привести эти столбцы к числовому типу

# Функция для преобразования столбцов, содержащих строку, к числовому типу 
def extract_numeric(x):    
    if type(x) == str:
        x = ''.join(filter(lambda y: str.isdigit(y) or y == '.', x.split()[0]))
        if x != '': 
            return float(x)
        else:
            return np.nan    
    elif type(x) == float:
        return x
    else:
        return np.nan

# столбцы, содержащие пропуски   
columns_with_nan = ['mileage', 'engine', 'max_power', 'torque', 'seats']
# список медиан, полученных из обучающего датасета
df_train_medians = []

# столбец 'seats' нам не нужен, поскольку он и так имеет тип float
for col in columns_with_nan[:-1]:    
    df_train[col] = df_train[col].apply(extract_numeric)
    df_train_medians.append(df_train[col].median())

# Дополним список медианой для столбца Seat
df_train_medians.append(df_train['seats'].median())

# Теперь очистим соответствующие столбцы тестового датасета от строковых составляющих 
for col in columns_with_nan[:-1]:    
    df_test[col] = df_test[col].apply(extract_numeric)

# Теперь заполним все пропуски медианными значениями из обучающего датасета
for i in range(len(columns_with_nan)):
    df_train[columns_with_nan[i]].fillna(df_train_medians[i], inplace=True)
    df_test[columns_with_nan[i]].fillna(df_train_medians[i], inplace=True)


/tmp/ipykernel_25423/3264144059.py:35: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_train[columns_with_nan[i]].fillna(df_train_medians[i], inplace=True)
/tmp/ipykernel_25423/3264144059.py:36: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(

In [20]:
X_train = df_train.drop('selling_price', axis=1)
y_train = df_train['selling_price']
X_test = df_test.drop('selling_price', axis=1)
y_test = df_test['selling_price']

In [26]:
# Определяем числовые и категориальные признаки
numeric_features = ['year', 'km_driven', 'mileage', 'engine', 'max_power', 'seats']
categorical_features = ['name', 'fuel', 'seller_type', 'transmission', 'owner']

# Создаем общий преобразователь
preprocessor = ColumnTransformer(
    transformers=[
        ('custom', CustomTransformer(), numeric_features + categorical_features),
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(drop='first', handle_unknown='ignore', dtype='byte'), categorical_features),
    ]
)

# Полный пайплайн с регрессором
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', Ridge(alpha=1))
])

# Обучение модели
pipeline.fit(df_train, y_train)

# Оценка
y_pred = pipeline.predict(df_test)
r2 = r2_score(y_test, y_pred)
print(f"R2 на тестовой выборке: {r2:.2f}")


ValueError: For a sparse output, all columns should be a numeric or convertible to a numeric.